In [15]:
import PyPDF2
import textract
import pickle
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn import metrics

warnings.filterwarnings("ignore")

In [16]:
def readPdf(name):
    text = ""
    
    try:
        pdf  = PyPDF2.PdfFileReader(open(name, 'rb'))
        for i in range(pdf.numPages):
            text += pdf.getPage(i).extractText()
    except:
        print('problem with ' + name)

    if text:
        return text
    else:
        return textract.process(name, method='tesseract').decode()

In [17]:
def process(x):
    return x.lower().translate(str.maketrans('', '', '0123456789_')).encode('ascii',errors='ignore').decode()

In [18]:
def classify_document(document, clf, bd = '', expediente=''):
    if type(bd) == pd.core.frame.DataFrame and not expediente:
        print('Expediente missing!')
        return
    else:
        text = readPdf(document)
        acao = clf.predict([text])[0] 
        print(acao)
        if type(bd) == pd.core.frame.DataFrame:
            if expediente in bd.index:
                bd.loc[expediente].text += text
                print('Expediente', expediente, 'updated!')
            else:
                bd.loc[expediente] = (acao, text)

In [19]:
def retrieval_classification(expediente, bd):
    print(bd.loc[str(expediente)])

In [20]:
def correct_classification(v, column, expediente, bd):
    bd.set_value(expediente, column, v)   

In [21]:
def retrain_model(clf, bd):
    x_train, x_test, y_train, y_test = train_test_split(bd.text, bd.acao, test_size = 0.2, random_state=42)
    clf = clf.fit(x_train, y_train)
    check_accuracy(clf, bd, True)

In [22]:
def check_accuracy(clf, bd, metric=False):
    x_train, x_test, y_train, y_test = train_test_split(bd.text, bd.acao, test_size = 0.2, random_state=42)
    predictions = clf.predict(x_test)
    
    if metric:
        print(metrics.classification_report(y_test, predictions))
        print(metrics.confusion_matrix(y_test, predictions))
    else:
        print(clf.score(x_test, y_test))

In [23]:
with open('../04_machine_learning/model.pkl', "rb") as f:
    clf = pickle.load(f)

with open('../04_machine_learning/dataset.pkl', "rb") as f:
    bd = pickle.load(f)

In [24]:
check_accuracy(clf, bd)

0.7424242424242424


In [25]:
correct_classification('50', 'acao', '02.000.00136/2017',bd)
bd.head()

,acao,text
expediente,,
02.000.00030/2017,9,PROCESSO:\nCLASSE:\nAUTOR:\nREU:\n\nPCTT: 92.1...
02.000.00035/2017,50,"\n\n \n\n \n\nPCTT 92,100.04\n\nURGENTE\n\n \..."
02.000.00136/2017,50,\n\nPODER JUDICIARIO\nTRIBUNAL REGIONAL FEDER...
02.000.00145/2017,43,03/02/2017Número: 1000055-73.2017.4.01.3200 ...
02.000.00186/2017,50,\n\nVara Unica\nSSJ Tefé\nFi.\n\nRubrica\n\n ...


In [26]:
%%time
retrain_model(clf, bd)

             precision    recall  f1-score   support

          1       1.00      1.00      1.00         2
         10       1.00      0.25      0.40         4
         14       0.00      0.00      0.00         5
         22       0.00      0.00      0.00         1
         28       0.00      0.00      0.00         2
         43       0.80      0.67      0.73         6
         47       0.44      0.53      0.48        15
         50       0.81      0.95      0.87        76
         60       0.00      0.00      0.00         1
         67       0.78      0.64      0.70        11
        227       0.67      1.00      0.80         2
        237       0.60      0.43      0.50         7

avg / total       0.70      0.75      0.71       132

[[ 2  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  1  1  0  1  0  0]
 [ 0  0  0  0  0  0  1  4  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  0  0  0  0]
 [ 0  0  0  0  0  4  2  0  0  0  0  0]
 [ 0  0  0  0  0  1  

In [27]:
%%time
text = readPdf('../01_source/Arquivos/02000000302017_6761978_Mandado_de_Intimacao_-_Proc_16374-70.2016.4.01.3200.pdf')
print(text[:1000])

PROCESSO:
CLASSE:
AUTOR:
REU:

PCTT: 92.100.04

    

See

PODER JUDICIARIO ZONA 2
JUSTIGA FEDERAL DE PRIMEIRO GRAU CENTRO
SEGAO JUDICIARIA DO ESTADO DO AMAZONAS
7? VARA FEDERAL

MANDADO DE INTIMAGAO

16374-70.2016.4.01.3200 OVAL O00 A AN
7100 - ACAO CIVIL PUBLICA

MINISTERIO PUBLICO FEDERAL

BANCO DO BRASIL S.A. E OUTROS.

MANDADO: N°/

INTIMAGAO DE :
CPFICNPY :
ENDEREGO:

FINALIDADE:

ADVERTENCIA:
ANEXO:

 

CAIXA ECONOMICA FEDERAL - CEF
00.360.305/0001-04
RUA RAMOS FERREIRA, 596, CENTRO, CEP: 69010-120, MANAUS/AM

INTIMAGAO PARA SE MANIFESTAR EM 72 (SETENTA E DUAS) HORAS, CONSOANTE
DESPACHO DE FLS, 42

PETICAO INICIAL E DESPACHO DE FLS, 42

SEDE DO JUIZO: 7 vana FEDERAL DA SEGAO JUDICIARIA DO ESTADO DO AMAZONAS

 

AVENIDA ANDRE ARAUJO - N° 25, EDIFICIO RIO NEGRO, 1° ANDAR
MANAUS-AM

CEP: 69.060-000

E-mail: 07vara.am@trf1 jus.br

Expedi este mandado por ordem deste Ju(zo Federal.

MANAUS, 16 de Janeiro de 2017,

 
     
 

DIEGO DO NAS NTO LIMA
Diretor(a) de Secretaria db(a) 7? VAR

In [28]:
%%time
clf.predict([text])[0]

CPU times: user 15.7 ms, sys: 0 ns, total: 15.7 ms
Wall time: 14.8 ms


9